In [18]:
%load_ext autoreload
%autoreload 2

import pandas as pd

df = pd.read_csv("responses.tsv", sep="\t")
df = pd.concat([df[df.user_choice == "Correct"], df[(df.suggested_label.notna()) & (df.user_choice != "Correct")]])
# df = df[(df.suggested_label.notna()) & (df.user_choice != "Correct")]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
df

,user,text,generated_label,user_choice,suggested_label,conversation_id
0,Carmen,Vertel iets in het Nederlands over Duitsland.,Factual,Correct,NaN,f729287836834ffa93383199fa3f4785
1,Carmen,смысл жизни,Opinion,Correct,NaN,5503ba243e8b4467b803d6d1c02ea85d
2,Carmen,"А напиши стилизацию для страницы, которую ты н...",Code,Correct,NaN,8d5a34c551a54091a80147a1e4602d3d
3,Carmen,The user is sending an instruction to a robot....,Code,Correct,NaN,a0df0440c8d94383b361dc680a222bcb
4,Carmen,"Do not generate ""As an AI language model"" use ...",Opinion,Correct,NaN,00ff24bd197c474d8a48bdf6a38807a6
...,...,...,...,...,...,...
119,gilad,ключевые идеи жиля де лёза,Factual,Incorrect,Multilingual,6f23427b199a43098df4e49ea3342d8e
120,gilad,was ist ein integer,Factual,Incorrect,Math,cc62dd47e7ba4fe7ab77f922b0e52436
131,gilad,你是谁,Factual,Incorrect,Multilingual,03e94df8fc1a43f98c48fd27128bce29
138,AB,Qual arquivo é usado pelo navegador para persi...,Factual,Incorrect,Multilingual,7859c868fa2442328a5e82586b416ab0


In [20]:
data = []

for _, row in df.iterrows():
    data.append(
        dict(
            prompt=row["text"],
            category=row["generated_label"] if row["user_choice"] == "Correct" else row["suggested_label"],
        )
    )

In [21]:
df = pd.DataFrame(data)

In [27]:
from ypl.pytorch.model.categorizer import CategorizerClassificationModel

categorizer = CategorizerClassificationModel.from_pretrained("deberta-base-v3-local").cuda()
categorizer.eval()
# categorizer.compile_cuda_graphs()

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


CategorizerClassificationModel(
  (category_model): DebertaV2ForSequenceClassification(
    (deberta): DebertaV2Model(
      (embeddings): DebertaV2Embeddings(
        (word_embeddings): Embedding(128100, 768, padding_idx=0)
        (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
        (dropout): StableDropout()
      )
      (encoder): DebertaV2Encoder(
        (layer): ModuleList(
          (0-11): 12 x DebertaV2Layer(
            (attention): DebertaV2Attention(
              (self): DisentangledSelfAttention(
                (query_proj): Linear(in_features=768, out_features=768, bias=True)
                (key_proj): Linear(in_features=768, out_features=768, bias=True)
                (value_proj): Linear(in_features=768, out_features=768, bias=True)
                (pos_dropout): StableDropout()
                (dropout): StableDropout()
              )
              (output): DebertaV2SelfOutput(
                (dense): Linear(in_features=768, out_features=

In [28]:
accuracy = []

for _, row in df.iterrows():
    accuracy.append(int(categorizer.categorize(row["prompt"])[0] == row["category"]))

print(sum(accuracy) / len(accuracy))

0.6436781609195402


In [17]:
accuracy = []

for _, row in df.iterrows():
    accuracy.append(int(categorizer.categorize(row["prompt"])[0] == row["category"]))

print(sum(accuracy) / len(accuracy))

0.17391304347826086


In [22]:
accuracy = []

for _, row in df.iterrows():
    accuracy.append(int(categorizer.categorize(row["prompt"])[0] == row["category"]))

print(sum(accuracy) / len(accuracy))

0.6091954022988506
